In [11]:
"""import modules"""
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets #to load a binary classification dataset
from sklearn.preprocessing import StandardScaler # get features
from sklearn.model_selection import train_test_split

In [12]:
# 0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale features
sc = StandardScaler() #make features with zero mean
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

"""converting to torch tensor"""
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

"""reshape data"""
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [13]:
# 1) Model
# Linear model f = wx + b , sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1) #get one output

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression(n_features)

In [14]:
# 2) Loss and optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

"""evaluate the model"""
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() #converting to class label
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

epoch: 10, loss = 0.4516
epoch: 20, loss = 0.3985
epoch: 30, loss = 0.3599
epoch: 40, loss = 0.3305
epoch: 50, loss = 0.3071
epoch: 60, loss = 0.2881
epoch: 70, loss = 0.2722
epoch: 80, loss = 0.2586
epoch: 90, loss = 0.2470
epoch: 100, loss = 0.2367
accuracy: 0.8860
